In [3]:
import math
import numpy as np
import tensorflow as tf
from env import Base2048Env
import os

env = Base2048Env()

env.reset()

Hi


(array([[0, 0, 0, 0],
        [2, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 2, 0]], dtype=int64), 0, False, {})

In [6]:
class Agent:
    def __init__(self, num_actions, state_size):
         
        initializer = tf.contrib.layers.xavier_initializer()
         
        self.input_layer = tf.placeholder(dtype=tf.float32, shape=[None, state_size])
         
        # Neural net starts here
         
        hidden_layer = tf.layers.dense(self.input_layer, 16, activation=tf.nn.leaky_relu, kernel_initializer=initializer)
        hidden_layer_2 = tf.layers.dense(hidden_layer, 16, activation=tf.nn.leaky_relu, kernel_initializer=initializer)
        hidden_layer_3 = tf.layers.dense(hidden_layer_2, 32, activation=tf.nn.leaky_relu, kernel_initializer=initializer)
         
        # Output of neural net
        out = tf.layers.dense(hidden_layer_3, num_actions, activation=None)
         
        self.outputs = tf.nn.softmax(out)
        self.choice = tf.argmax(self.outputs, axis=1)
         
        # Training Procedure
        self.rewards = tf.placeholder(shape=[None, ], dtype=tf.float32)
        self.actions = tf.placeholder(shape=[None, ], dtype=tf.int32)
         
        one_hot_actions = tf.one_hot(self.actions, num_actions)
         
        cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=out, labels=one_hot_actions)
         
        self.loss = tf.reduce_mean(cross_entropy * self.rewards)
         
        self.gradients = tf.gradients(self.loss, tf.trainable_variables())
         
        # Create a placeholder list for gradients
        self.gradients_to_apply = []
        for index, variable in enumerate(tf.trainable_variables()):
            gradient_placeholder = tf.placeholder(tf.float32)
            self.gradients_to_apply.append(gradient_placeholder)
         
        # Create the operation to update gradients with the gradients placeholder.
        optimizer = tf.train.AdamOptimizer()
        self.update_gradients = optimizer.apply_gradients(zip(self.gradients_to_apply, tf.trainable_variables()))


In [7]:
# TODO Create the discounted and normalized rewards function
discount_rate = 0.99


def discount_normalize_rewards(rewards):
    discounted_rewards = np.zeros_like(rewards)
    total_rewards = 0

    for i in reversed(range(len(rewards))):
        total_rewards = total_rewards * discount_rate + rewards[i]
        discounted_rewards[i] = total_rewards

    return discounted_rewards


def format_state(state):
    return [math.log(number, 2) if number != 0 else 0 for number in list(np.reshape(state, 16))]

print(format_state(np.array(
    [
        [0, 0, 0, 0],
        [2, 2, 2, 2],
        [4, 4, 4, 4],
        [64, 64, 64, 64]
    ]
)))


[0, 0, 0, 0, 1.0, 1.0, 1.0, 1.0, 2.0, 2.0, 2.0, 2.0, 6.0, 6.0, 6.0, 6.0]


In [ ]:
# TODO Create the training loop
tf.reset_default_graph()

# Modify these to match shape of actions and states in your environment
num_actions = 4
state_size = 16

path = "./2048-pg/"

training_episodes = 10001

episode_batch_size = 1

loading = True

agent = Agent(num_actions, state_size)

init = tf.global_variables_initializer()

saver = tf.train.Saver(max_to_keep=2)

if not os.path.exists(path):
    os.makedirs(path)

with tf.Session() as sess:
    if not loading:
        sess.run(init)
    else:
        checkpoint = tf.train.get_checkpoint_state(path)
        saver.restore(sess, checkpoint.model_checkpoint_path)

    total_episode_rewards = []

    # Create a buffer of 0'd gradients
    gradient_buffer = sess.run(tf.trainable_variables())
    for index, gradient in enumerate(gradient_buffer):
        gradient_buffer[index] = gradient * 0

    for episode in range(1, training_episodes):

        state = format_state(env.reset()[0])

        episode_history = []
        episode_rewards = 0
        
        done = False
        moves = 0
        while not done:
            moves += 1
            # Get weights for each action
            action_probabilities = sess.run(agent.outputs, feed_dict={agent.input_layer: [state]})
            action_choice = np.random.choice(range(num_actions), p=action_probabilities[0])

            state_next, reward, done, _ = env.step(action_choice)
            episode_history.append([state, action_choice, reward, state_next])
            state = format_state(state_next)

            episode_rewards += reward

            if done:
                total_episode_rewards.append(episode_rewards)
                episode_history = np.array(episode_history)
                episode_history[:, 2] = discount_normalize_rewards(episode_history[:, 2])

                ep_gradients = sess.run(agent.gradients, feed_dict={agent.input_layer: np.vstack(episode_history[:, 0]),
                                                                    agent.actions: episode_history[:, 1],
                                                                    agent.rewards: episode_history[:, 2]})
                # add the gradients to the grad buffer:
                for index, gradient in enumerate(ep_gradients):
                    gradient_buffer[index] += gradient
                #env.render()
                #print('\nTotal Moves: {}'.format(moves))
                break

        if episode % episode_batch_size == 0:

            feed_dict_gradients = dict(zip(agent.gradients_to_apply, gradient_buffer))

            sess.run(agent.update_gradients, feed_dict=feed_dict_gradients)

            for index, gradient in enumerate(gradient_buffer):
                gradient_buffer[index] = gradient * 0

        if episode % 1000 == 0:
            saver.save(sess, path + "pg-checkpoint", episode)
            print("Average reward / 100 eps: " + str(np.mean(total_episode_rewards[-100:])))

    print("TOTAL AVERAGE REWARD:" + str(np.mean(total_episode_rewards)))
    print("HIGH SCORE: "+str(np.max(total_episode_rewards)))


INFO:tensorflow:Restoring parameters from ./2048-pg/pg-checkpoint-1000


Average reward / 100 eps: 1797.56


Average reward / 100 eps: 1965.28


Average reward / 100 eps: 1776.68


Average reward / 100 eps: 1901.44


Average reward / 100 eps: 1894.92


Average reward / 100 eps: 1926.52


Average reward / 100 eps: 1569.08
